# 확률 네트워크기반 친환경 자동차 충전소(GVCS) 로케이션 인텔리전스(Location Intelligence) 대시보드

## 제안 배경

**친환경 자동차 생산에만 관심이 쏠려 사용자를 고려하지 못한 설치로 인한 잉여 충전소 발생**

- 민관 협력을 통해 충전 서비스 관련 사업이 꾸준히 증가함. 하지만 전국에 설치된 공공 급속충전기 총 2,896개(2021년 기준) 중 하루 1회 이하로 사용한 충전기가 1,164개로 전체의 40.1%를 차지하고 있다. 심지어 17개는 전혀 사용하지 않은 것으로 나타남.

- 이와 같은 충전소마다 이용 편차가 큰 이유는 전기차 보급 목표에 따라 구역별로  충전소를 나눠 설치했기 때문이라는 지적도 있다.

- 한국전기차충전협동조합은 앞으로 수요가 몰리는 거점을 중심을 중심으로 충전기 설치를 해야하 한다는 의견 주장함.

- 기존의 정부 중심으로 공급자 입장에서 설치하던 충전소를 친환경 자동차 차주를 고려해 이용자 중심으로 변화 설치가 필요하다는 지적도 있다.

즉, **꾸준히 증가하는 친환경 차량의 수요를 위해 저비용 고효율의 충전소 설치를 목표 한다.** 충전소 설치 사업자의 기대 수익 최적화를 위해 사용자 입장을 고려한 체계적이며 최적의 입지 선정 결과를 제시함.

## 사용 데이터

1. 충전소 위치 및 현황 정보
2. 급속충전기 보급 현황
3. LPG충전소 현황
4. 오존 월별 대기오염도
5. 아황산가스 월별 대기오염도
6. 이산화질소 월별 대기오염도
7. 미세먼지 월별 대기오염도
8. 행정구역별 인구수
9. 항만 현황
10. 지역별 전기차 등록대 수

In [679]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import networkx as nx
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

## 상위 요소 - 환경적 요인

대기오염도를 기반하여 확률 값을 계산한다.

### 오존 월별 대기오염도 데이터 불러오기

In [680]:
ozone_file_path = "C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\오존_월별_도시별_대기오염도.csv"
df_ozone = pd.read_csv(ozone_file_path, encoding='cp949')
df_ozone

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.029,0.034,0.043,0.042,0.044,0.033
1,서울특별시,서울특별시,0.022,0.025,0.037,0.036,0.041,0.038
2,부산광역시,부산광역시,0.031,0.036,0.043,0.041,0.041,0.028
3,대구광역시,대구광역시,0.025,0.032,0.043,0.040,0.045,0.031
4,인천광역시,인천광역시,0.026,0.030,0.040,0.042,0.044,0.041
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.034,0.039,0.050,0.048,0.051,0.038**
168,경상남도,도평균,0.032,0.037,0.044,0.044,0.045,0.031
169,제주특별자치도,제주특별자치도,0.044,0.046,0.051,0.051,0.045,0.029
170,제주특별자치도,서귀포,0.045,0.048,0.054,0.052,0.046,0.028


In [681]:
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [682]:
ozen_col =  df_ozone.iloc[:,2:-1].columns.tolist()

In [683]:
df_ozone.drop(columns=ozen_col, inplace=True)

In [684]:
def advanced_replace(df, col, str, regex):
    '''
    DataFrame 특정 열의 특수문자 제거
    :param df: 대상 DataFrame
    :param col: column
    :param str: 제거할 특수 문자.
    :param regex: 적용할 정규표현식
    :return:
    '''
    df[col] = df[col].replace(str,0)
    df[col] = df[col].replace(to_replace=regex, value=r'', regex=True)
    return df

In [685]:
# 해당 컬럼 float 타입으로 변경
advanced_replace(df_ozone, df_ozone.iloc[:,2:].columns.tolist(), '-', r'[^0-9.0-9]')
df_ozone['2021.07'] = df_ozone['2021.07'].astype(float)
df_ozone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [686]:
def show_norm(mean,std,min,max):
     '''
     정규분포 함수를 matplotlib을 통해 시각화
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     '''
     min = min - std
     # x값 설정(min에서 max까지 unit만큼씩 증가하는 x)
     x = np.linspace(min,max,100)
     # y값은 평균이 mean이고 표준편차가 std인 확률밀도함수
     fig = px.line(x=x,y=norm.pdf(x,loc=mean,scale=std))
     fig.show()

In [687]:
mean = df_ozone.iloc[:,2].mean()
std = df_ozone.iloc[:,2].std()
min = df_ozone.iloc[:,2].min()
max = df_ozone.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 오존의 누적확률 구하기

In [688]:
def cal_norm(mean,std,min,max,value):
     '''
     정규분포에서 value에 대한 누적확률 구하기
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     value: 누적확률을 구하고자 하는 데이터의 측정값
     '''
     min = min - std
     red ='rgb(239,85,59)'
     blue = 'rgb(100,110,250)'
     fig = go.Figure()
     
     # 확률 값을 구할 특정 구간의 범위 설정
     cum_a = np.linspace(min,value,100)
     cum_b = np.linspace(value,max,100)
     
     pro = norm(mean,std).cdf(value)-norm(mean,std).cdf(min).round(3)
     # 구간 사이에 색을 채움
     fig.add_trace(go.Scatter(x=cum_a,y=norm.pdf(cum_a,mean,std), fill = 'tozeroy',name='적합',text=pro,line=dict(color=blue)))
     fig.add_trace(go.Scatter(x=cum_b,y=norm.pdf(cum_b,mean,std), fill = 'tozeroy',name='부적합',text=pro,line=dict(color=red)))
     annotations =[]
     annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(pro,3), font=dict(size=15,color=blue), xshift=-40, yshift=-100, bordercolor=blue,borderwidth=2))
     annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(1-pro,3),font=dict(size=15,color=red),xshift=40, yshift=-100, bordercolor=red,borderwidth=2))
     fig.update_layout(annotations=annotations)
     # value까지의 누적확률에서 min까지의 누적확률을 뺌
     fig.show()

     # 최종 누적확률 반환
     return pro

In [689]:
busan_oz = df_ozone[df_ozone['구분(2)']=='부산광역시'].loc[2,'2021.07']
ozone_pro = cal_norm(mean,std,min,max,busan_oz)
ozone_pro

0.23346014669602194

### 아황산가스 월별 데이터 불러오기

In [690]:
so2_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\아황산가스_월별_도시별_대기오염도.csv'
df_so2 = pd.read_csv(so2_file_path, encoding='cp949')
df_so2

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.003,0.003,0.003,0.003,0.003,0.003
1,서울특별시,서울특별시,0.003,0.003,0.003,0.003,0.003,0.003
2,부산광역시,부산광역시,0.004,0.003,0.003,0.002,0.003,0.003
3,대구광역시,대구광역시,0.002,0.002,0.002,0.002,0.002,0.002
4,인천광역시,인천광역시,0.004,0.003,0.003,0.003,0.003,0.003
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.003,0.003,0.003,0.003,0.003,0.003**
168,경상남도,도평균,0.003,0.003,0.003,0.003,0.003,0.003
169,제주특별자치도,제주특별자치도,0.002,0.001,0.002,0.002,0.002,0.002
170,제주특별자치도,서귀포,0.001,0.001,0.001,0.001,0.001,0.001


In [691]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [692]:
so2_col = df_so2.iloc[:,2:-1].columns.tolist()

In [693]:
df_so2.drop(columns=so2_col, inplace=True)
df_so2

,구분(1),구분(2),2021.07
0,총계,소계,0.003
1,서울특별시,서울특별시,0.003
2,부산광역시,부산광역시,0.003
3,대구광역시,대구광역시,0.002
4,인천광역시,인천광역시,0.003
...,...,...,...
167,경상남도,합천,0.003**
168,경상남도,도평균,0.003
169,제주특별자치도,제주특별자치도,0.002
170,제주특별자치도,서귀포,0.001


In [694]:
advanced_replace(df_so2,'2021.07','-',r'[^0-9.0-9]')
df_so2['2021.07'] = df_so2['2021.07'].astype(float)

In [695]:
df_so2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [696]:
df_so2.describe()

,2021.07
count,172.000000
mean,0.002674
std,0.000764
min,0.001000
25%,0.002000
50%,0.003000
75%,0.003000
max,0.005000


In [697]:
mean = df_so2.iloc[:,2].mean()
std = df_so2.iloc[:,2].std()
min = df_so2.iloc[:,2].min()
max = df_so2.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 아황산가스의 누적확률 구하기

In [698]:
busan_so2 = df_so2[df_so2['구분(2)']=='부산광역시'].loc[2,'2021.07']
so2_pro = cal_norm(mean,std,min,max,busan_so2)
so2_pro

0.6640367362231872

### 이산화질소 월별 데이터 불러오기

In [699]:
no2_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\이산화질소_월별_도시별_대기오염도.csv'
df_no2 = pd.read_csv(no2_file_path, encoding='cp949')
df_no2

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.019,0.019,0.014,0.012,0.011,0.010
1,서울특별시,서울특별시,0.030,0.032,0.022,0.019,0.018,0.016
2,부산광역시,부산광역시,0.018,0.017,0.015,0.014,0.014,0.010
3,대구광역시,대구광역시,0.022,0.018,0.013,0.012,0.010,0.009
4,인천광역시,인천광역시,0.024,0.028,0.021,0.017,0.017,0.014
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.014,0.011,0.010,0.009,0.010,0.009**
168,경상남도,도평균,0.016,0.014,0.013,0.011,0.012,0.009
169,제주특별자치도,제주특별자치도,0.011,0.011,0.010,0.010,0.009,0.006
170,제주특별자치도,서귀포,0.007,0.007,0.006,0.006,0.005,0.004


In [700]:
no2_col = df_no2.iloc[:,2:-1].columns.tolist()

In [701]:
df_no2.drop(columns=no2_col, inplace=True)

In [702]:
advanced_replace(df_no2,'2021.07','-',r'[^0-9.0-9]')
df_no2['2021.07'] = df_no2['2021.07'].astype(float)

In [703]:
df_no2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [704]:
df_no2.describe()

,2021.07
count,172.000000
mean,0.007610
std,0.003294
min,0.003000
25%,0.005000
50%,0.007000
75%,0.009250
max,0.019000


In [705]:
mean = df_no2.iloc[:,2].mean()
std = df_no2.iloc[:,2].std()
min = df_no2.iloc[:,2].min()
max = df_no2.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 이산화질소 누적확률 구하기

In [706]:
busan_no2 = df_no2[df_no2['구분(2)']=='부산광역시'].loc[2,'2021.07']
no2_pro = cal_norm(mean,std,min,max,busan_no2)

### 일산화탄소 월별 데이터 불러오기

In [707]:
co_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\일산화탄소_월별_도시별_대기오염도.csv'
df_co = pd.read_csv(co_file_path, encoding='cp949')
df_co

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,0.5,0.5,0.4,0.4,0.4,0.3
1,서울특별시,서울특별시,0.6,0.5,0.4,0.4,0.4,0.3
2,부산광역시,부산광역시,0.4,0.4,0.3,0.3,0.3,0.2
3,대구광역시,대구광역시,0.5,0.4,0.3,0.4,0.4,0.3
4,인천광역시,인천광역시,0.6,0.6,0.4,0.4,0.4,0.4
...,...,...,...,...,...,...,...,...
167,경상남도,합천,0.4,0.3,0.3,0.3,0.3,0.2**
168,경상남도,도평균,0.5,0.4,0.4,0.4,0.4,0.3
169,제주특별자치도,제주특별자치도,0.3,0.3,0.3,0.3,0.3,0.2
170,제주특별자치도,서귀포,0.3,0.3,0.3,0.3,0.2,0.2


In [708]:
co_col = df_co.iloc[:,2:-1].columns.tolist()

In [709]:
df_co.drop(columns=co_col, inplace=True)

In [710]:
advanced_replace(df_co, '2021.07','-',r'[^0-9.0-9]')
df_co['2021.07'] = df_co['2021.07'].astype(float)

In [711]:
df_co.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [712]:
df_co.describe()

,2021.07
count,172.000000
mean,0.328488
std,0.084825
min,0.100000
25%,0.300000
50%,0.300000
75%,0.400000
max,0.600000


In [713]:
mean = df_co.iloc[:,2].mean()
std = df_co.iloc[:,2].std()
min = df_co.iloc[:,2].min()
max = df_co.iloc[:,2].max()
show_norm(mean,std,min,max)

### 부산광역시 일산화탄소 누적확률 구하기

In [714]:
busan_co = df_co[df_co['구분(2)']=='부산광역시'].loc[2,'2021.07']
co_pro = cal_norm(mean,std,min,max,busan_co)

### 미세먼지 월별 데이터 불러오기

In [715]:
pm25_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\미세먼지_PM2.5__월별_도시별_대기오염도.csv'
df_pm25 = pd.read_csv(pm25_file_path, encoding='cp949')
df_pm25

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,25,27,17,19,19,12
1,서울특별시,서울특별시,29,32,19,20,19,15
2,부산광역시,부산광역시,21,21,15,16,15,9
3,대구광역시,대구광역시,24,23,17,18,20,10
4,인천광역시,인천광역시,28,33,19,20,23,19
...,...,...,...,...,...,...,...,...
167,경상남도,합천,18,17,14,15,18,11**
168,경상남도,도평균,19,19,15,15,16,12*
169,제주특별자치도,제주특별자치도,18,24,14,16,14,8
170,제주특별자치도,서귀포,16,23,14,16,12,8


In [716]:
df_pm25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구분(1)    172 non-null    object
 1   구분(2)    172 non-null    object
 2   2021.02  172 non-null    object
 3   2021.03  172 non-null    object
 4   2021.04  172 non-null    object
 5   2021.05  172 non-null    object
 6   2021.06  172 non-null    object
 7   2021.07  172 non-null    object
dtypes: object(8)
memory usage: 10.9+ KB


In [717]:
pm25_col = df_pm25.iloc[:,2:-1].columns.tolist()

In [718]:
df_pm25.drop(columns=pm25_col, inplace=True)

In [719]:
advanced_replace(df_pm25, '2021.07','-',r'[^0-9.0-9]')
df_pm25['2021.07'] = df_pm25['2021.07'].astype(float)

In [720]:
df_pm25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [721]:
df_pm25.describe()

,2021.07
count,172.000000
mean,11.558140
std,2.888188
min,0.000000
25%,10.000000
50%,11.000000
75%,13.000000
max,19.000000


In [722]:
mean = df_pm25.iloc[:,2].mean()
std = df_pm25.iloc[:,2].std()
min = df_pm25.iloc[:,2].min()
max = df_pm25.iloc[:,2].max()
show_norm(mean,std,min,max)

In [723]:
busan_pm25 = df_pm25[df_pm25['구분(2)']=='부산광역시'].loc[2,'2021.07']
pm25_pro = cal_norm(mean,std,min,max,busan_pm25)

In [724]:
pm10_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\미세먼지_PM10__월별_도시별_대기오염도.csv'
df_pm10 = pd.read_csv(pm10_file_path, encoding='cp949')
df_pm10

,구분(1),구분(2),2021.02,2021.03,2021.04,2021.05,2021.06,2021.07
0,총계,소계,43,65,41,52,32,22
1,서울특별시,서울특별시,48,67,42,61,33,24
2,부산광역시,부산광역시,37,60,37,36,26,18
3,대구광역시,대구광역시,41,59,42,41,35,19
4,인천광역시,인천광역시,46,69,42,59,36,29
...,...,...,...,...,...,...,...,...
167,경상남도,합천,29,44,32,36,27,17**
168,경상남도,도평균,33,53,34,36,27,20
169,제주특별자치도,제주특별자치도,36,76,37,44,27,17
170,제주특별자치도,서귀포,33,68,35,40,23,21


In [725]:
pm10_col = df_pm10.iloc[:,2:-1].columns.tolist()

In [726]:
df_pm10.drop(columns=pm10_col, inplace=True)

In [727]:
advanced_replace(df_pm10, '2021.07', '-', r'[^0-9.0-9]')
df_pm10['2021.07'] = df_pm10['2021.07'].astype(float)

In [728]:
df_pm10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   구분(1)    172 non-null    object 
 1   구분(2)    172 non-null    object 
 2   2021.07  172 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.2+ KB


In [729]:
df_pm10.describe()

,2021.07
count,172.000000
mean,20.750000
std,4.725352
min,0.000000
25%,18.000000
50%,20.000000
75%,23.000000
max,36.000000


In [730]:
mean = df_pm10.iloc[:,2].mean()
std = df_pm10.iloc[:,2].std()
min = df_pm10.iloc[:,2].min()
max = df_pm10.iloc[:,2].max()
show_norm(mean,std,min,max)

In [731]:
busan_pm10 = df_pm10[df_pm10['구분(2)']=='부산광역시'].loc[2,'2021.07']
pm10_pro = cal_norm(mean,std,min,max,busan_pm10)

## 상위 요소 - 사회적 요인

### 행정구역 인구 데이터 불러오기

In [768]:
people_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\행정구역_시군구_별__성별_인구수.csv'
df_people = pd.read_csv(people_file_path, encoding='cp949', header=1)
df_people

,행정구역(시군구)별,총인구수 (명)
0,전국,51592660
1,서울특별시,9500480
2,종로구,144005
3,중구,122087
4,용산구,222009
...,...,...
287,거창군,60756
288,합천군,42613
289,제주특별자치도,677413
290,제주시,493076


In [733]:
df_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   행정구역(시군구)별  292 non-null    object
 1   총인구수 (명)    292 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [734]:
df_people.describe()

,총인구수 (명)
count,2.920000e+02
mean,5.614314e+05
std,3.176068e+06
min,0.000000e+00
25%,5.282050e+04
50%,1.935545e+05
75%,3.709285e+05
max,5.159266e+07


### 부산광역시 인구 데이터 추출

In [775]:
df_busan_people = df_people[27:44]
df_busan_people = df_busan_people.reset_index()
df_busan_people.drop(columns='index',inplace=True)
df_busan_people

,행정구역(시군구)별,총인구수 (명)
0,부산광역시,3340214
1,중구,40585
2,서구,103961
3,동구,88257
4,영도구,109407
5,부산진구,351022
6,동래구,273976
7,남구,260617
8,북구,280852
9,해운대구,393130


In [ ]:
cal_norm(df_busan_people.mean(), df_busan_people.std(), df_busan_people.min(), df_busan_people.max(), df_busan_people[df_busan_people['행정구역(시군구)별']=='사하구'])

### 유동인구 데이터 불러오기

In [735]:
foottraffic_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 유동인구 데이터.csv'
df_foottraffic = pd.read_csv(foottraffic_file_path, encoding='cp949')
df_foottraffic

,구군,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,월 평균
0,강서구,"325,284.39","323,049.87","335,120.06","340,570.52","332,289.77","340,678.45","337,488.64","328,374.74","328,764.38","333,414.26","340,405.10","337,753.15","333,599.44"
1,금정구,"482,598.89","477,047.11","484,669.59","490,864.46","487,435.45","484,936.00","483,898.58","482,964.23","487,082.06","484,633.90","485,502.59","488,392.99","485,002.15"
2,기장군,"341,181.17","343,398.79","342,489.52","345,184.20","348,793.68","348,500.32","350,763.40","349,328.59","348,077.97","348,449.77","345,905.89","344,245.29","346,359.88"
3,남구,"539,072.58","526,693.91","524,611.67","524,961.53","526,758.54","531,078.89","529,810.42","530,199.16","531,658.35","525,870.37","528,106.40","531,610.18","529,202.67"
4,동구,"185,466.33","185,459.30","189,082.61","192,702.25","189,819.14","191,618.73","189,864.16","189,368.74","190,328.67","191,182.03","194,732.77","193,341.86","190,247.22"
5,동래구,"553,655.76","549,718.60","543,091.96","535,439.53","533,908.61","528,124.07","527,730.64","525,468.79","532,656.70","527,498.20","520,126.56","524,788.88","533,517.36"
6,부산진구,"757,261.10","760,742.04","760,212.63","761,728.33","765,460.09","764,636.01","759,638.81","754,837.78","763,942.81","761,731.75","763,818.34","768,957.14","761,913.90"
7,북구,"559,133.62","554,946.17","543,084.57","543,479.25","543,145.91","540,381.96","542,289.98","546,905.11","551,394.74","542,887.88","539,732.97","545,952.52","546,111.22"
8,사상구,"496,102.34","492,871.78","493,054.71","496,932.63","494,094.59","494,207.66","492,973.61","490,296.85","493,842.47","491,808.21","493,815.86","495,297.44","493,774.85"
9,사하구,"641,541.35","632,045.91","628,461.73","633,805.98","630,333.83","629,635.14","630,260.69","631,377.03","633,402.47","627,551.97","625,380.25","631,475.21","631,272.63"


In [736]:
foottraffic_col = df_foottraffic.iloc[:,1:-1].columns.tolist()

In [737]:
df_foottraffic.drop(columns=foottraffic_col, inplace=True)
df_foottraffic

,구군,월 평균
0,강서구,"333,599.44"
1,금정구,"485,002.15"
2,기장군,"346,359.88"
3,남구,"529,202.67"
4,동구,"190,247.22"
5,동래구,"533,517.36"
6,부산진구,"761,913.90"
7,북구,"546,111.22"
8,사상구,"493,774.85"
9,사하구,"631,272.63"


In [738]:
advanced_replace(df_foottraffic, '월 평균','-',r'[^0-9.0-9]')
df_foottraffic['월 평균'] = df_foottraffic['월 평균'].astype(float)

In [739]:
df_foottraffic.describe()

,월 평균
count,16.000000
mean,441344.991250
std,205145.538509
min,121290.850000
25%,311221.577500
50%,440973.920000
75%,536665.825000
max,867841.630000


In [740]:
# 부산 행정구 단위 유동인구 정규분포
mean = df_foottraffic.iloc[:,1].mean()
std = df_foottraffic.iloc[:,1].std()
min = df_foottraffic.iloc[:,1].min()
max = df_foottraffic.iloc[:,1].max()
show_norm(mean,std,min,max)

In [741]:
# 사하구 유동인구 확률 밀도 계산 
busan_foottraffic = df_foottraffic[df_foottraffic['구군']=='사하구'].iloc[0,1]
foottraffic_pro = cal_norm(mean,std,min,max,busan_foottraffic)
foottraffic_pro


0.8177299761126527

### 친환경 자동차 등록수 데이터 불러오기

In [742]:
vehicle_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\연료별 자동차 등록 수.csv'
df_vehicle = pd.read_csv(vehicle_file_path, encoding='cp949')
drop_col = df_vehicle.iloc[:,1:3].columns.tolist()
df_vehicle.drop(columns=drop_col,inplace=True)
df_vehicle

,연료,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,계
0,휘발유,"1,649,518","714,221","584,780","818,874","312,081","341,412","297,231","94,929","3,056,798","365,724","403,958","525,588","398,182","466,016","654,776","886,361","302,434","11,872,883"
1,경유,"1,077,183","550,411","449,206","627,029","266,237","247,724","217,553","66,126","2,450,214","371,277","380,402","516,001","430,676","547,999","664,042","736,135","253,205","9,851,420"
2,엘피지,"240,940","111,371","106,654","113,654","88,883","67,456","36,101","12,985","404,449","58,335","70,304","89,210","93,615","111,494","127,818","139,592","58,412","1,931,273"
3,전기,"45,133","16,059","18,076","15,191","6,233","9,550","3,948","2,180","50,111","9,117","10,421","12,142","8,931","10,658","13,018","14,957","26,931","272,656"
4,하이브리드(휘발유+전기),"147,022","61,167","48,320","90,275","24,091","24,167","19,593","11,034","256,643","24,001","25,842","32,669","25,448","38,154","36,999","56,219","30,057","951,701"
5,하이브리드(경유+전기),"2,656","4,187","2,146","7,672",464,393,194,141,"3,964",286,580,461,694,486,405,"1,774",341,"26,844"
6,하이브리드(LPG+전기),"1,601",735,806,696,628,481,457,153,"3,453",611,639,728,785,833,974,"1,034",335,"14,949"
7,기타연료,"7,172","16,097","4,545","13,915","4,036","3,830","5,205","1,208","29,809","4,376","6,918","9,574","5,182","9,666","10,974","9,161",886,"142,554"
8,수소,"2,435","1,473",352,"1,139","1,018","1,045","2,395",205,"4,004","1,959","1,095",908,"1,435",448,77,"1,899",5,"21,892"
9,총합,"3,173,660","1,475,721","1,214,885","1,688,445","703,671","696,058","582,677","188,961","6,259,445","835,686","900,159","1,187,281","964,948","1,185,754","1,509,083","1,847,132","672,606","25,086,172"


In [743]:
# 전국 자동차 수 통계값 구하기 (전기, 수소)
col = df_vehicle.iloc[:,1:].columns.tolist()
for c in col:
    advanced_replace(df_vehicle,c,'-',r'[^0-9.0-9]')
    df_vehicle[c] = df_vehicle[c].astype(float)
df_elec_vehicle = df_vehicle.iloc[3:7,:-1].sum()
df_elec_vehicle = df_elec_vehicle[1:]

df_elec_vehicle = df_elec_vehicle.astype(float)
df_elec_vehicle.describe()


count        17.000000
mean      74479.411765
std       75151.087588
min       13508.000000
25%       34591.000000
50%       50131.000000
75%       73984.000000
max      314171.000000
dtype: float64

In [744]:
df_hidro_vehicle = df_vehicle.iloc[8,1:-1]
df_hidro_vehicle = df_hidro_vehicle.astype(float)
df_hidro_vehicle.describe()

count      17.000000
mean     1287.764706
std      1030.779531
min         5.000000
25%       448.000000
50%      1095.000000
75%      1899.000000
max      4004.000000
Name: 8, dtype: float64

In [745]:
# 전국 전기차 정규분포
show_norm(df_elec_vehicle.mean(), df_elec_vehicle.std(), df_elec_vehicle.min(), df_elec_vehicle.max())
show_norm(df_hidro_vehicle.mean(), df_hidro_vehicle.std(), df_hidro_vehicle.min(), df_hidro_vehicle.max())

In [746]:
df_hidro_vehicle

서울    2435.0
부산    1473.0
대구     352.0
인천    1139.0
광주    1018.0
대전    1045.0
울산    2395.0
세종     205.0
경기    4004.0
강원    1959.0
충북    1095.0
충남     908.0
전북    1435.0
전남     448.0
경북      77.0
경남    1899.0
제주       5.0
Name: 8, dtype: float64

In [747]:
num_elec = df_elec_vehicle['부산']
num_hidro = df_hidro_vehicle['부산']
elec_vehicle_pro = cal_norm(mean,std,min,max,num_elec)
print(elec_vehicle_pro)
hidro_vehicle_pro = cal_norm(mean,std,min,max,num_hidro)
print(hidro_vehicle_pro)

0.034978348735681564


0.011008641890079197


### LPG 충전소 현황 데이터 불러오기 (수소차)

In [748]:
LPG_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 LPG 충전소 현황(한국가스안전공사).csv'
df_LPG = pd.read_csv(LPG_file_path, encoding='cp949')
df_LPG

,행정구역,업소명,소재지,관리구분,위도,경도
0,부산 강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594
1,부산 강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891
2,부산 강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260
3,부산 강서구,(주)지원네트웍스 부산강서LPG충전소,부산 강서구 낙동북로 310,자동차충전,35.212507,128.962572
4,부산 강서구,신명지에너지,부산 강서구 르노삼성대로 560,자동차충전,35.097554,128.927351
...,...,...,...,...,...,...
58,부산 연제구,부산개인택시엘피지충전소,부산 연제구 월드컵대로 399,자동차충전,35.186013,129.055974
59,부산 영도구,(주)태종에너지,부산 영도구 영선대로 41,자동차충전,35.084996,129.040033
60,부산 영도구,부산에너지,부산 영도구 해양로 160,용기+자동차+탱크+13kg용기,35.091275,129.069484
61,부산 영도구,개인택시 영도LPG충전소,부산 영도구 해양로 296,자동차충전,35.082176,129.076429


In [749]:
df_LPG.shape[0]

63

### 전기차 충전소 현황 데이터 불러오기

In [750]:
EVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 전기차 충전소 정보.csv'
df_EVCS = pd.read_csv(EVCS_file_path, encoding='cp949')
df_EVCS

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,위도,경도
0,환경부(한국자동차환경협회),kbs 부산방송총국,1,DC차데모+AC3상+DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
1,환경부(한국자동차환경협회),kbs 부산방송총국,11,DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
2,환경부(한국자동차환경협회),kbs 부산방송총국,12,DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
3,환경부(한국자동차환경협회),국립부산과학관,11,DC콤보,부산광역시,기장군,부산광역시 기장군 기장읍 석산리 293,35.204002,129.212987
4,환경부(한국자동차환경협회),국립부산과학관,12,DC콤보,부산광역시,기장군,부산광역시 기장군 기장읍 석산리 293,35.204002,129.212987
...,...,...,...,...,...,...,...,...,...
6291,한국전자금융,부산 플러스타워,1,AC완속,부산광역시,강서구,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),35.093685,128.905372
6292,한국전자금융,부산 플러스타워,2,AC완속,부산광역시,강서구,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),35.093685,128.905372
6293,한국전자금융,부산 휴마트괴정점,1,AC완속,부산광역시,사하구,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),35.103200,129.002528
6294,한국전자금융,부산 휴마트괴정점,2,AC완속,부산광역시,사하구,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),35.103200,129.002528


In [751]:
# 부산 전체 충전기 수 구하기
df_EVCS.shape[0]

6296

### 교통 편의성 데이터

### 주변 충전소와의 거리 데이터

### 고속도로 여부

## 상위 요소 - 경제적 요소

### 충전소 설치 비용 불러오기 (전기차)

### 충전소 설치 비용 데이터 불러오기 (수소차) 

### 총 유지비용

### 주차 구획 수 데이터 불러오기

In [752]:
parking_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산광역시 공영주차장 정보 조회.csv'
df_parking = pd.read_csv(parking_file_path, encoding='cp949')
df_parking

,운영형태,관리기관명,주차장명,주차장관리번호,소재지지번주소,소재지도로명주소,전화번호,주차장유형,주차구획수,평일운영시작시각,...,부제시행구분,운영요일,요금정보,주차기본시간,추가단위시간,추가단위요금,1일주차권요금적용시간,결제방법,특기사항,실시간주차면수
0,구(군)직영,부산광역시 사상구청,감전동주변,2010000013,-,사상구 감전동 589-2,051-310-4501,노외,26,0:00,...,-,-,-,10,10,0,-,-,-,-
1,구(군)직영,부산광역시 사상구청,장성주차장,2010000015,-,사상구 주례2동 92-278,051-310-4501,노외,16,0:00,...,-,-,-,10,10,0,-,-,-,-
2,민간위탁,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23 (엄궁동 47),-,051-310-4503,노외,93,0:00,...,-,-,-,10,10,100,-,-,-,-
3,구(군)직영,부산광역시 사상구청,일신주택주변,2011000003,-,사상구 주례3동 545-3,051-310-4501,노외,10,0:00,...,-,-,-,10,10,0,-,-,-,-
4,구(군)직영,부산광역시 사상구청,주례중학교주변,2012000004,-,사상구 주례2동 82-13,051-310-4501,노외,8,0:00,...,-,-,-,10,10,0,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,구(군)직영,부산광역시 기장군청,정관읍 상업지역 노상주차장,2016000017,-,기장군 정관면 732,-,노상,46,0:00,...,-,-,-,10,10,0,-,-,-,-
611,민간운영,부산광역시 기장군청,1구역 2소,2008000002,-,기장군 기장읍 280-2,051-792-4719,노상,12,9:00,...,-,-,-,10,10,200,-,-,-,-
612,민간운영,부산광역시 기장군청,1구역 3소,2008000003,-,기장군 기장읍 444-1,051-792-4719,노상,13,9:00,...,-,-,-,10,10,200,-,-,-,-
613,민간운영,부산광역시 기장군청,3구역 1소,2008000006,-,기장군 기장읍 418-12,051-792-4719,노상,24,9:00,...,-,-,-,10,10,200,-,-,-,-


### 주차 구획수 통계 구하기

In [753]:
df_parking_area = df_parking['주차구획수'].astype(int)
df_parking_area.describe()

count    615.000000
mean      62.378862
std       70.372193
min       -4.000000
25%       18.000000
50%       38.000000
75%       79.000000
max      414.000000
Name: 주차구획수, dtype: float64

In [758]:
show_norm(df_parking_area.mean(),df_parking_area.std(),df_parking_area.min(),df_parking_area.max())
a = df_parking_area.iloc[610]
cal_norm(df_parking_area.mean(),df_parking_area.std(),df_parking_area.min(),df_parking_area.max(), a)

0.381979238207332

## 상위 요소 - 기술적 요소

### 복합 충전소 여부 (수소차)

### 충전시간 (전기차)
 급속의 경우는 50분으로 "적합" 또는 "True" 의 비율을 100%로 설정
 완속의 경우는 